In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

/home/lior/git/theory


In [2]:
#Import general libraries (needed for functions)`
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from gatelevel_qiskit.waveform_comparator import WaveformComparator

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb
from qiskit.pulse import Schedule, Play, Gaussian, DriveChannel, ShiftPhase, Waveform, ControlChannel, MeasureChannel, AcquireChannel
from qiskit.circuit import Gate
#Import Qiskit classes 
import qiskit
from pprint import pprint
from deepdiff import DeepDiff

from gatelevel_qiskit.circuit_to_qua import CircuitQuaTransformer
from gatelevel_qiskit.lib import wfs_no_samples, summary_of_inst, get_min_time
from gatelevel_qiskit.simple_backend import simple_backend
from rb_config import config_base


In [3]:
# generate RB 1QB
c1 = qiskit.circuit.quantumcircuit.QuantumCircuit(1)
c1.x(0)
rb_circs1, xdata = rb.randomized_benchmarking_seq(length_vector=[1,2,3,4,5], nseeds=3, rb_pattern=[[0]])
circ = rb_circs1[0][4]

In [4]:
circ.draw()

┌───┐┌───┐┌───┐ ░ ┌─────┐┌───┐ ░ ┌───┐┌───┐┌───┐ ░ ┌───┐┌───┐┌───┐┌───┐»
qr_0: ┤ Y ├┤ H ├┤ S ├─░─┤ SDG ├┤ H ├─░─┤ Y ├┤ H ├┤ S ├─░─┤ Z ├┤ S ├┤ H ├┤ S ├»
      └───┘└───┘└───┘ ░ └─────┘└───┘ ░ └───┘└───┘└───┘ ░ └───┘└───┘└───┘└───┘»
cr: 1/═══════════════════════════════════════════════════════════════════════»
                                                                             »
«       ░ ┌───┐┌───┐ ░ ┌───┐┌───┐┌─┐
«qr_0: ─░─┤ Z ├┤ H ├─░─┤ H ├┤ S ├┤M├
«       ░ └───┘└───┘ ░ └───┘└───┘└╥┘
«cr: 1/═══════════════════════════╩═
«                                 0

In [5]:
circ_qua = CircuitQuaTransformer(pulse_backend=simple_backend,
                                config_base=config_base,
                                circuit=circ)

In [6]:
print(circ_qua.to_qua())

with program() as prog:
    I = [None] * 1
    I[0] = declare(fixed)
    align(*['d0', 'm0'])
    play('wf_Y', 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    play('wf_X90', 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(-1.5707963267948966, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    play('wf_X90', 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    play('wf_Y', 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    play('wf_X90', 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(3.141592653589793, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    play('wf_X90', 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    frame_rotation(3.141592653589793, 'd0')
    frame_rotation(1.5707963267948966, 'd0')
    play('wf_X90', 'd0')
    frame_rotati

In [7]:
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm import SimulationConfig
from qm.qua import *

qmm = QuantumMachinesManager('52.29.8.241')
qm = qmm.open_qm(circ_qua.config)

exec(circ_qua.to_qua())
job = qm.simulate(prog, SimulationConfig(500, include_analog_waveforms=True))
print(job.id())
wfs_sim = job.simulated_analog_waveforms()['controllers']['con1']['ports']

wfs_circ = circ_qua.to_waveforms(get_min_time(wfs_sim))

comp = WaveformComparator(wfs_sim, wfs_circ)
pprint(comp)
assert len(comp.diff) == 0

2020-11-11 15:22:19,103 - qm - INFO - Performing health check
2020-11-11 15:22:19,250 - qm - INFO - Health check passed
2020-11-11 15:22:19,357 - qm - WARNING - Client's version (0.5.157) does not match the server version (0.5.140).
2020-11-11 15:22:19,487 - qm - INFO - Flags: 
2020-11-11 15:22:19,487 - qm - INFO - Executing high level program
1605100939468
{}
